In [2]:
import requests as req
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup as bsp
from selenium import webdriver
from seleniumwire import webdriver  # blinker == 1.7.0
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent
# from googlesearch import search
from datetime import datetime
from collections import defaultdict

base = 'https://www.amazon.com'
thirdWheel = '/s?k='
date_pattern = r"\b(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}\b"
number_pattern = r"\d+"

To be Run Yet

In [3]:
import pandas as pd

# Read the CSV file, skipping the header
file_path = "AmazonCategory.csv"
df = pd.read_csv(file_path, delimiter=";", header=None, skiprows=1)

# Create the dictionary with subcategories as keys and categories as values
category = {}

for _, row in df.iterrows():
    dawcategory, subcategory = row[0], row[1]
    
    if pd.notna(subcategory):  # Ignore empty subcategories
        category[subcategory] = dawcategory

# Print the dictionary
for subcategory, dawcategory in category.items():
    print(f"Subcategory: {subcategory} → Category: {dawcategory}")


Subcategory: Cooking+Appliances → Category: Appliances
Subcategory: Dishwashers → Category: Appliances
Subcategory: Laundry+Appliances → Category: Appliances
Subcategory: Parts++Accessories → Category: Appliances
Subcategory: Ranges+Ovens++Cooktops → Category: Appliances
Subcategory: Refrigerators+Freezers++Ice+Makers → Category: Appliances
Subcategory: Arts+Crafts++Sewing+Storage → Category: Arts+Crafts++Sewing
Subcategory: Beading++Jewelry+Making → Category: Arts+Crafts++Sewing
Subcategory: Craft++Hobby+Fabric → Category: Arts+Crafts++Sewing
Subcategory: Craft+Supplies++Materials → Category: Arts+Crafts++Sewing
Subcategory: Fabric+Decorating → Category: Arts+Crafts++Sewing
Subcategory: Knitting++Crochet+Supplies → Category: Arts+Crafts++Sewing
Subcategory: Model++Hobby+Building → Category: Arts+Crafts++Sewing
Subcategory: Needlework+Supplies → Category: Arts+Crafts++Sewing
Subcategory: Painting+Drawing++Art+Supplies → Category: Arts+Crafts++Sewing
Subcategory: Printmaking+Supplies → 

In [ ]:
# session = requests.Session()
# session.headers.update({"User-Agent": UserAgent().random})
driver = webdriver.Chrome()
# driver = webdriver.Firefox(service = service, options = options)
driver.get(base)
#   It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10) 
html = driver.page_source
driver.quit()          
soup = bsp(html, 'lxml')
boxes = soup.find_all('a', class_ = 'a-link-normal _fluid-quad-image-label-v2_style_centerImage__30wh- aok-block image-window')
print(boxes)
links = []
urls = []
categ = []
for box in boxes:
    actual = box.get('href')
    url = base + actual
    print("Url : ", url)
    urls.append(url)
    som = box.get('aria-label')
    link = som.replace(' ', '+')
    link = base + thirdWheel + link
    print("Links : ", link)
    links.append(link)
    categ.append(som)
    print(som)


In [4]:
urls = {}
for cat in category:
    url = base+thirdWheel+cat
    urls[url] = category[cat]
print(urls)


{'https://www.amazon.com/s?k=Cooking+Appliances': 'Appliances', 'https://www.amazon.com/s?k=Dishwashers': 'Appliances', 'https://www.amazon.com/s?k=Laundry+Appliances': 'Appliances', 'https://www.amazon.com/s?k=Parts++Accessories': 'Appliances', 'https://www.amazon.com/s?k=Ranges+Ovens++Cooktops': 'Appliances', 'https://www.amazon.com/s?k=Refrigerators+Freezers++Ice+Makers': 'Appliances', 'https://www.amazon.com/s?k=Arts+Crafts++Sewing+Storage': 'Arts+Crafts++Sewing', 'https://www.amazon.com/s?k=Beading++Jewelry+Making': 'Arts+Crafts++Sewing', 'https://www.amazon.com/s?k=Craft++Hobby+Fabric': 'Arts+Crafts++Sewing', 'https://www.amazon.com/s?k=Craft+Supplies++Materials': 'Arts+Crafts++Sewing', 'https://www.amazon.com/s?k=Fabric+Decorating': 'Arts+Crafts++Sewing', 'https://www.amazon.com/s?k=Knitting++Crochet+Supplies': 'Arts+Crafts++Sewing', 'https://www.amazon.com/s?k=Model++Hobby+Building': 'Arts+Crafts++Sewing', 'https://www.amazon.com/s?k=Needlework+Supplies': 'Arts+Crafts++Sewing',

In [ ]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data


In [ ]:
def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [5]:
products = []
# categories = []
cat = 0
me = False
# proxies = get_valid_proxies()

for url, cat in urls.items():
    total_samples = 0
    myurl = url
    while myurl:
        # proxy = proxies[total_samples % len(proxies)] if proxies else None
        options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
        print("VISITING : ", myurl)
        # sdsd = Options()
        # sdsd.add_argument("--headless")  # Run in background
        # sdsd.add_argument("--disable-gpu")
        # sdsd.add_argument("--no-sandbox")
        # sdsd.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(seleniumwire_options = options)
        # driver = webdriver.Firefox()
        driver.get(myurl)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(10)
        html = driver.page_source
        driver.quit()
        # headers = rotate_user_agent(proxy)
        # response = req.get(myurl, headers = headers)
        # session = requests.Session()
        # session.headers.update({"User-Agent": UserAgent().random})
        # response = req.get(myurl, headers = {"User-Agent": UserAgent().random})
        # if response.status_code == 200:
            # soup = bsp(response.text, 'lxml')
        soup = bsp(html, 'lxml')
        # print(soup)
        productes = soup.find_all('a', class_ = 'a-link-normal s-no-outline')
        if not productes:
            productes = soup.find_all('a', class_ = 'a-link-normal s-line-clamp-4 s-link-style a-text-normal')
        if not productes:
            productes = soup.find_all('a' , class_= 'a-link-normal s-line-clamp-2 s-link-style a-text-normal')
        print(productes)
        for product in productes:
            product = base + product.get('href')
            products.append(product)
            print(product)
        csv_file = 'Amazon.csv'
        cat = cat.replace('+', " ")
        info = {"Products": products, "Categories": cat}
        new_df = pd.DataFrame(info)
        if not me:
            new_df.to_csv(csv_file, mode='a', index = False)
            me = True
        else:
            new_df.to_csv(csv_file, mode='a', header=False, index = False)
        print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")
        
        total_samples += 1
        # Reset lists to prevent duplicates in the next iteration
        products.clear()
        nextP = soup.find('a', class_ = 's-pagination-item s-pagination-next s-pagination-button s-pagination-button-accessibility s-pagination-separator')
        if nextP:
            myurl = base + nextP.get('href')
        else:
            break
    cat += 1



VISITING :  https://www.amazon.com/s?k=Cooking+Appliances


KeyboardInterrupt: 

In [ ]:

csv_file = 'Amazon.csv'
meta = 'Products.csv'
rev = 'Reviews.csv'
df = pd.read_csv(csv_file)
samples = 0
me = False
for _,row in df.iterrows():
    agent = {
        "link" : "",
        "date" : "",
        "main_category" : "",
        "title" : "",
        "average_rating" : 0.0,
        "rating_number" : 0,
        "features":[],
        "description":[],
        "price" : "",
        "images":[],
        "videos":[],
        "store" : "",
        "categories" : "",
        "details":{},
        "parent_asin" : "",
        "bought_together": []
    }

    comment = {
        "link" : "",
        "rating" : 0.0,
        "title" : "",
        "text" : "",
        "images" : "",
        "asin" : "",
        "parent_asin" : "",
        "user_id" : "",
        "timestamp" : "",
        "verified_purchase" : 0,
        "helpful_vote" : ""
    }
    print("\n\n\n\n\n\n-------------------------------------------------------------------Product-------------------------------------------------------------------")
    options = {
            'headers' :{
                "User-Agent":UserAgent().random
            }
        }
    link = row['Products']
    print(link)
    driver = webdriver.Chrome(seleniumwire_options = options)
    driver.get(link)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(10)
    html = driver.page_source
    driver.quit()
    soup = bsp(html, 'lxml')

    feat = []
    img = []
    vid = []
    det = []

    agent["link"] = link


    # extracting Category
    category = row['Categories']
    print("Category :", category)
    agent["main_category"] = category

    # extracting Title
    try:
        titleBoard = soup.find('span', id = 'productTitle')
        title = titleBoard.text.strip()
    except:
        title = ""
    print("Title :", title)
    agent["title"] = title

    # extracting ratings
    try:
        ratingBoard = soup.find('div', id = 'averageCustomerReviews')
        # extracting average rating
        avgRatingBoard = ratingBoard.find('span', class_ = 'a-size-base a-color-base')
        avgRating = avgRatingBoard.text.strip()
        # extracting rating number
        ratingNoBoard = ratingBoard.find('span', id = 'acrCustomerReviewText')
        ratingNo = ratingNoBoard.text.strip()
        junk = " ratings"
        ratingNo = ratingNo.replace(junk, "")
    except:
        avgRating = ""
        ratingNo = ""
    print("Average Rating :", avgRating)
    agent["average_rating"] = avgRating
    print("Rating :", ratingNo)
    agent["rating_number"] = ratingNo

    # extracting price
    try:
        priceBoard = soup.find('div', class_ = 'a-section a-spacing-none aok-align-center aok-relative').find('span', class_ = 'aok-offscreen')
        price = priceBoard.text.strip()[0:5]
    except:
        price = ""
    print("Price :", price)
    agent["price"] = price

    # exracting media
    try:
        mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro regularAltImageViewLayout')
        if not mediaBoard:
            mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro gridAltImageViewLayoutIn1x7')
            if not mediaBoard:
                mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-extra-large regularAltImageViewLayout')
        media = mediaBoard.find_all('img')
        images = [img['src'] for img in media]
        vids = images[-1]
        images = images[0:-1]
    except:
        images = []
        vids = []
    print("Images :", images)
    agent["images"] = images   
    print("Videos :", vids)
    agent["videos"] = vids

    # extracting the store
    try:
        storeBoard = soup.find('tr', class_ = 'a-spacing-small po-brand').find('td', class_ = 'a-span9')
        store = storeBoard.text.strip()
    except:
        try:
            storeBoard = soup.find('a', id = 'bylineInfo').text.strip()
            junk = "Visit the "
            store = storeBoard.replace(junk, "")
        except:
            store = ""
    print("Store :", store)
    agent["store"] = store

    # extracting features
    try:
        featuresBoard = soup.find('ul', class_ = 'a-unordered-list a-vertical a-spacing-mini').find_all('li')
        features = [feature.text.strip() for feature in featuresBoard]
    except:
        features = []
    print("Features :", features)
    agent["features"] = features
        
    # extracting details
    try:
        detailsBoard = soup.find('div', class_ = 'a-expander-content a-expander-section-content a-section-expander-inner').find('table', class_ = 'a-keyvalue prodDetTable').find_all('tr')
    except:
        try:
            detailsBoard = soup.find('div', id = 'detailBullets_feature_div').find_all('li')
        except:
            try:
                detailsBoard = soup.find('table', id = 'productDetails_detailBullets_sections1').find_all('tr')
            except:   
                try:
                    detailsBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list').find_all('tr')
                except:
                    detailsBoard = []

    # find th for the dictionary subject and td for the disctionary details
    details = {}
    if detailsBoard:
        for detailBoard in detailsBoard:
            detailingH = detailBoard.find('th').text.strip() if detailBoard.find('th') else None
            detailingA = detailBoard.find('td').text.strip() if detailBoard.find('td') else None
            if detailingH and detailingA:
                detailed = {detailingH: detailingA} 
                details |= detailed  
    else:
        details = {}
    print("Details :", details)
    agent["details"] = details
                    
    # extracting asin
    try:
        asin = details['ASIN']
    except:
        asin = ""
    print("ASIN :", asin)
    agent["parent_asin"] = asin

    try:
        date = details['Date First Available']
    except:
        date = ""
    print("Date :", date)
    agent["date"] = date
    
    # extracting description
    try:
        descriptionBoard = soup.find('table', class_ = 'a-normal a-spacing-micro')
        if not descriptionBoard:
            descriptionBoard = soup.find('ul', id = 'a-nostyle').find_all('div', class_ = 'a-fixed-left-grid-col a-col-right')
        descriptions = descriptionBoard.find_all('td', class_ = 'a-span9')
        description = [dec.text.strip() for dec in descriptions]
    except:
        description = []
    print("Description :", description)
    agent["description"] = description

    # extracting bought together
    try:
        boughtTogetherBoard = soup.find('div', id = 'a-cardui _c3AtZ_new-thumbnail-box_1W9Ku _c3AtZ_two-item-thumbnail-box_7kF95')
        if not boughtTogetherBoard:
            boughtTogetherBoard = soup.find('div', class_ = 'a-cardui _p13n-desktop-sims-fbt_fbt-desktop_new-thumbnail-box__36bD3 _p13n-desktop-sims-fbt_fbt-desktop_two-item-thumbnail-box__jV2am')
        if not boughtTogetherBoard:
            boughtTogetherBoard = soup.find('div', class_ = 'a-cardui _p13n-desktop-sims-fbt_fbt-desktop_new-thumbnail-box__36bD3')
        boughtTogetherS = boughtTogetherBoard.find_all('a', class_ = 'a-link-normal a-text-normal', href = True)
        if not boughtTogetherS:
            boughtTogetherS = boughtTogetherBoard.find_all('a', class_ = 'a-link-normal', href = True)        
        boughtTogether = [base + link['href'] for link in boughtTogetherS]
    except:
        boughtTogether = []
    print("Bought Together :", boughtTogether)
    agent["bought_together"] = boughtTogether

    # extracting further categories
    try:
        categoriesBoard = soup.find_all('div', id= 'tp-inline-twister-dim-values-container')
        categoriesBoardAh = categoriesBoard[-1].find_all('li')
        categories = [categoriesBoardh.text.strip() for categoriesBoardh in categoriesBoardAh]
    except:
        categories = []
    print("Categories :", categories)
    agent["categories"] = categories


    ############################################################################################################################################################
    print("\n\n\n-------------------------------------------------------------------Comments-------------------------------------------------------------------")
    # extracting reviews
    reviewsUS = soup.find('ul', id = 'cm-cr-dp-review-list').find_all('li')
    reviewsGlobal = soup.find('ul', id = 'cm-cr-global-review-list').find_all('li')
    reviews = reviewsUS + reviewsGlobal
    for review in reviews:
        # extracting rating
        try:
            reviewRatingBoard = review.find('i', {'data-hook': 'review-star-rating'}).find('span', class_ = 'a-icon-alt')
            reviewRating = reviewRatingBoard.text[0:3]
        except:
            reviewRating = ""
        print("Rating :", reviewRating)
        comment["rating"].append(reviewRating)

        # extracting title
        try:
            reviewTitleBoard = review.find('a', class_ = 'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold')
            reviewTitles = reviewTitleBoard.find_all('span')
            reviewTitle = reviewTitles[-1].text.strip()
        except:
            reviewTitle = ""
        print("Title :", reviewTitle)
        comment["title"].append(reviewTitle)

        # extracting Text
        try:
            reviewTextBoard = review.find('div', class_ = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content')
            reviewText = reviewTextBoard.text.strip()
        except:
            reviewText = ""
        print("Review :", reviewText)
        comment["text"].append(reviewText)

        # extracting images
        try:
            reviewImages = review.find('div', class_ = 'review-image-tile-section').find_all('img')
            reviewhttps://www.amazon.com/sspa/click?ie=UTF8&spc=MToyODMyMzI0OTcxNTk0OToxNzQxMDg4MDY2OnNwX210ZjozMDAxMjU4NzQ4Njg2MDI6OjA6Og&url=%2FGlo-Skin-Beauty-Crayon-Trademark%2Fdp%2FB00UIBKNSA%2Fref%3Dsr_1_78_sspa%3Fdib%3DeyJ2IjoiMSJ9.c49SQ5UI-wcu0l24kOdoqyJaYURqL9ddFz9Sv8JNEe82KySS-7h5ilJ-PXedBuHJBtZ27wB7foASamagcsdvAa9a3ucXmVeIE1KEbuv_NKkwg_8SAWpgrerWvzdSi2-_6UAGo2SfvvlHAn-W_aW3howS1jwN07DOgCRO-BBlvPEmIN3YzL3YdcywY6y38fDv8XAMYqhZHnbYFBZdz-DsrxP210wNzusp5IBflwGVaTZEvY1drKN4td8Zxxv_R64YMp80YfGRaPieqY0PAzr5XmkFgoYLgCh9y_d9rNcSuiE.C_t_cqqWRcrnVxDB-sWrzMJOMs1B7VKMNjoMnhJItTo%26dib_tag%3Dse%26keywords%3DAll%2BBeauty%26qid%3D1741088066%26sr%3D8-78-spons%26xpid%3D7NFhI7LAUx9Mh%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9tdGY%26psc%3D1,All+Beauty
Images = [img['src'] for img in reviewImages]
        except:
            reviewImages = []
        print("Images :", reviewImages)
        comment["images"].append(reviewImages)

        # extracting ASIN
        reviewAsin = review.get('id')
        print("ASIN :", reviewAsin)
        comment["asin"].append(reviewAsin)

        # extracting parent ASIN
        reviewParentAsin = asin
        print("Parent ASIN :", reviewParentAsin)
        comment["parent_asin"].append(review)

        # eaxtracting username
        try:
            username = review.find('span', class_ = 'a-profile-name').text.strip()
        except:
            username = ""
        print("Username :", username)
        comment["user_id"].append(username)

        # extracting date
        try:
            reviewTimeBoard = review.find('span', class_ = 'a-size-base a-color-secondary review-date')
            reviewTimes = reviewTimeBoard.text
            reviewTimes = re.search(date_pattern, reviewTimes)
            reviewTiming = datetime.strptime(reviewTimes, "%B %d, %Y")
            reviewTime = int(reviewTiming.timestamp())
        except:
            reviewTime = ""
        print("Date :", reviewTime)
        comment["timestamp"].append(reviewTime)

        verified = False
        if review.find('span', class_ = 'a-size-mini a-color-state a-text-bold'):
            verified = True
        print("Verified :", verified)
        comment["verified_purchase"].append(verified)

        # extracting helpfulness
        try:
            reviewHelpful = review.find('span', class_ = 'a-size-base a-color-tertiary cr-vote-text')
            reviewHelpful = reviewHelpful.text
            haha = "One"
            helpful = 0
            Helpfulness = re.search(number_pattern, reviewHelpful)
            if Helpfulness:
                helpful = int(Helpfulness.group())
            elif haha in reviewHelpful:
                helpful = 1
        except:
            helpful = 0
        print("Helpful :", helpful)
        comment["helpful_vote"].append(helpful)

    myMeta = pd.DataFrame(agent)
    myRev = pd.DataFrame(comment)
    if not me:
        myMeta.to_csv(meta, mode='a', index = False)
        myRev.to_csv(rev, mode='a', index = False)
        me = True
    else:
        myMeta.to_csv(meta, mode='a', header=False, index = False)
        myRev.to_csv(rev, mode='a', header=False, index = False)
    
    print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")
    





In [ ]:


# Define the date strings
date_str1 = "June 22, 2016"
date_str2 = "September 30, 2023"

# Convert the date strings to datetime objects
date1 = datetime.strptime(date_str1, "%B %d, %Y")
date2 = datetime.strptime(date_str2, "%B %d, %Y")

# Compare the dates
if date1 > date2:
    print(f"{date_str1} is greater than {date_str2}")
else:
    print(f"{date_str1} is not greater than {date_str2}")

In [ ]:
# Date string in the format "Month Day, Year"
date_str = "January 17, 2025"

# Parse the string into a datetime object
date_obj = datetime.strptime(date_str, "%B %d, %Y")

# Convert to Unix time
unix_time = int(date_obj.timestamp())

print(f"Unix Time: {unix_time}")



In [ ]:
# Example statement
statement = "The item costs 700 dollars, and the discount is."

# Regular expression to match one or more digits
number_pattern = r"\d+"

# Find the first number in the statement
match = re.search(number_pattern, statement)

if match:
    number = int(match.group())  # Convert the matched number to an integer
    print(f"Extracted number: {number}")
else:
    print("No number found.")
